In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import missingno as msno
pd.options.display.max_columns = 200
pd.options.display.max_rows = 100
sns.set_theme(style="whitegrid")

In [2]:
#chargement des bases de données
data_customers = pd.read_csv('olist_customers_dataset.csv')
data_seller = pd.read_csv('olist_sellers_dataset.csv')
data_geo = pd.read_csv('olist_geolocation_dataset.csv')

data_product = pd.read_csv('olist_products_dataset.csv')
data_categorie = pd.read_csv('product_category_name_translation.csv')

data_item = pd.read_csv('olist_order_items_dataset.csv')
data_payment_order = pd.read_csv('olist_order_payments_dataset.csv')
data_reviews_order = pd.read_csv('olist_order_reviews_dataset.csv')
data_orders = pd.read_csv('olist_orders_dataset.csv')

In [3]:
data_O=data_customers.groupby('customer_unique_id', as_index=False).agg(nco=('customer_id','count'))
data_O['nco'].unique()

array([ 1,  2,  3,  4,  6,  7,  5,  9, 17])

In [4]:
data_payment_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [5]:
data_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [6]:
#on extrait les custumers id et customers unique id
data_cu_id=data_customers.drop(columns=['customer_zip_code_prefix','customer_city','customer_state'])
data_cu_id

,customer_id,customer_unique_id
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066
...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e


In [7]:
data_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [8]:
#transformation des variable de object à datetime
data_orders['order_purchase_timestamp']=pd.to_datetime(data_orders['order_purchase_timestamp'], 
                                                       format='%Y-%m-%d %H:%M:%S')
data_orders['order_approved_at']=pd.to_datetime(data_orders['order_approved_at'], 
                                                format='%Y-%m-%d %H:%M:%S')
data_orders['order_delivered_carrier_date']=pd.to_datetime(data_orders['order_delivered_carrier_date'], 
                                                           format='%Y-%m-%d %H:%M:%S')
data_orders['order_delivered_customer_date']=pd.to_datetime(data_orders['order_delivered_customer_date'], 
                                                            format='%Y-%m-%d %H:%M:%S')
data_orders['order_estimated_delivery_date']=pd.to_datetime(data_orders['order_estimated_delivery_date'], 
                                                            format='%Y-%m-%d %H:%M:%S')

In [9]:
data_ord=data_orders.drop(columns=['order_approved_at','order_delivered_carrier_date',
                                   'order_delivered_customer_date','order_estimated_delivery_date'])

data_ord=data_ord.join(data_cu_id.set_index('customer_id'), on='customer_id')
data_ord_it=data_item.groupby('order_id', as_index=False).agg(n_article=('order_id','count'),
                                                              price=('price','sum'),
                                                              fright_value = ('freight_value','sum')
                                                              )
data_ord=data_ord.join(data_ord_it.set_index('order_id'), on='order_id')
data_ord.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  99441 non-null  object        
 1   customer_id               99441 non-null  object        
 2   order_status              99441 non-null  object        
 3   order_purchase_timestamp  99441 non-null  datetime64[ns]
 4   customer_unique_id        99441 non-null  object        
 5   n_article                 98666 non-null  float64       
 6   price                     98666 non-null  float64       
 7   fright_value              98666 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 6.1+ MB


In [10]:
data_ord[(data_ord['order_status']=='unavailable')&(data_ord['n_article'].isnull())]

,order_id,customer_id,order_status,order_purchase_timestamp,customer_unique_id,n_article,price,fright_value
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,41fc647b8c6bd979b1b6364b60471b50,NaN,NaN,NaN
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,0e634b16e4c585acbd7b2e8276ce6677,NaN,NaN,NaN
687,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,596ed6d7a35890b3fbac54ec01f69685,NaN,NaN,NaN
737,d57e15fb07fd180f06ab3926b39edcd2,470b93b3f1cde85550fc74cd3a476c78,unavailable,2018-01-08 19:39:03,2349bbb558908e0955e98d47dacb7adb,NaN,NaN,NaN
1160,2f634e2cebf8c0283e7ef0989f77d217,7353b0fb8e8d9675e3a704c60ca44ebe,unavailable,2017-09-27 20:55:33,21c933c8dd97d088e64c50988c90ccf5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
99147,6cf5c427fb11bc5fe8abefa86aa7ba0b,91ac14a1e50bc022eeecf0c2ba37006a,unavailable,2017-10-17 10:53:48,9004096d8ffc515d4189e661fc4bcb84,NaN,NaN,NaN
99233,1c7c8ab324927f5b6e52338bd110bef4,95b1767df51d6fa09402267d8de314df,unavailable,2017-12-23 18:48:09,0aa3b096dc796c251f9b36a822051ed6,NaN,NaN,NaN
99252,aaab15da689073f8f9aa978a390a69d1,df20748206e4b865b2f14a5eabbfcf34,unavailable,2018-01-16 14:27:59,a33e0969408919ba06779f497ead93ec,NaN,NaN,NaN
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,e72a90a2b29fe1a8795b284aaaa3246f,NaN,NaN,NaN


In [11]:
data_item[data_item['order_id']=='c272bcd21c287498b4883c7512019702']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value


In [12]:
data_ord.drop(columns=['customer_id'],inplace=True)
data_ord

,order_id,order_status,order_purchase_timestamp,customer_unique_id,n_article,price,fright_value
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,1.0,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,af07308b275d755c9edb36a90c618231,1.0,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,3a653a41f6f9fc3d2a113cf8398680e8,1.0,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977,1.0,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,72632f0f9dd73dfee390c9b22eb56dd6,1.0,19.90,8.72
...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,delivered,2017-03-09 09:54:05,6359f309b166b0196dbf7ad2ac62bb5a,1.0,72.00,13.08
99437,63943bddc261676b46f01ca7ac2f7bd8,delivered,2018-02-06 12:58:58,da62f9e57a76d978d02ab5362c509660,1.0,174.90,20.10
99438,83c1379a015df1e13d02aae0204711ab,delivered,2017-08-27 14:46:43,737520a9aad80b3fbbdad19b66b37b30,1.0,205.99,65.02
99439,11c177c8e97725db2631073c19f07b62,delivered,2018-01-08 21:28:27,5097a5312c8b157bb7be58ae360ef43c,2.0,359.98,81.18


In [13]:
data_payment_order[data_payment_order['order_id']=='c272bcd21c287498b4883c7512019702']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
3956,c272bcd21c287498b4883c7512019702,1,credit_card,1,97.68


In [14]:
data_po=data_payment_order.groupby('order_id', as_index=False).agg(total_price=('payment_value','sum'))
data_ord=data_ord.join(data_po.set_index('order_id'), on='order_id')

In [15]:
max(data_orders['order_purchase_timestamp'])

Timestamp('2018-10-17 17:30:18')

In [16]:
#initialisation des dates pour les plages de validation
date_11=pd.to_datetime('2016-09-04 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_12=pd.to_datetime('2018-03-04 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_21=pd.to_datetime('2016-09-19 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_22=pd.to_datetime('2018-03-19 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_31=pd.to_datetime('2016-10-04 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_32=pd.to_datetime('2018-04-04 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_41=pd.to_datetime('2016-10-19 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_42=pd.to_datetime('2018-04-19 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_51=pd.to_datetime('2016-11-04 3:00:00', format='%Y-%m-%d %H:%M:%S')
date_52=pd.to_datetime('2018-05-04 3:00:00', format='%Y-%m-%d %H:%M:%S')

In [17]:
#initialisation des plages de données
data_1=data_ord[data_ord['order_purchase_timestamp'].between(date_11,date_12,inclusive="neither")]
data_2=data_ord[data_ord['order_purchase_timestamp'].between(date_21,date_22,inclusive="neither")]
data_3=data_ord[data_ord['order_purchase_timestamp'].between(date_31,date_32,inclusive="neither")]
data_4=data_ord[data_ord['order_purchase_timestamp'].between(date_41,date_42,inclusive="neither")]
data_5=data_ord[data_ord['order_purchase_timestamp'].between(date_51,date_52,inclusive="neither")]

In [18]:
data_1

,order_id,order_status,order_purchase_timestamp,customer_unique_id,n_article,price,fright_value,total_price
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,1.0,29.99,8.72,38.71
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977,1.0,45.00,27.20,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,72632f0f9dd73dfee390c9b22eb56dd6,1.0,19.90,8.72,28.62
5,a4591c265e18cb1dcee52889e2d8acc3,delivered,2017-07-09 21:57:05,80bb27c7c16e8f973207a5086ab329e2,1.0,147.90,27.36,175.26
6,136cce7faa42fdb2cefd53fdc79a6098,invoiced,2017-04-11 12:22:08,36edbb3fb164b1f16485364b6fb04c73,1.0,49.90,16.05,65.95
...,...,...,...,...,...,...,...,...
99435,880675dff2150932f1601e1c07eadeeb,delivered,2017-02-23 09:05:12,831ce3f1bacbd424fc4e38fbd4d66d29,1.0,139.90,16.09,155.99
99436,9c5dedf39a927c1b2549525ed64a053c,delivered,2017-03-09 09:54:05,6359f309b166b0196dbf7ad2ac62bb5a,1.0,72.00,13.08,85.08
99437,63943bddc261676b46f01ca7ac2f7bd8,delivered,2018-02-06 12:58:58,da62f9e57a76d978d02ab5362c509660,1.0,174.90,20.10,195.00
99438,83c1379a015df1e13d02aae0204711ab,delivered,2017-08-27 14:46:43,737520a9aad80b3fbbdad19b66b37b30,1.0,205.99,65.02,271.01


In [19]:
#test pour extraire les features essentielle à mon application
data_ord_cus_1=data_1.groupby('customer_unique_id', as_index=False).agg(n_achats=('order_id','count'),
                                                                        fright_value =('fright_value','mean'),
                                                                        price = ('price','mean'),
                                                                        total_price = ('total_price','mean'),
                                                                        n_article =('n_article','mean'),
                                                                        first=('order_purchase_timestamp','min'),
                                                                        last=('order_purchase_timestamp','max')
                                                                        )
data_ord_cus_1

,customer_unique_id,n_achats,fright_value,price,total_price,n_article,first,last
0,0000f46a3911fa3c0805444483337064,1,17.22,69.00,86.22,1.0,2017-03-10 21:05:03,2017-03-10 21:05:03
1,0000f6ccb0745a6a4b88665a16c9f078,1,17.63,25.99,43.62,1.0,2017-10-12 20:29:41,2017-10-12 20:29:41
2,0004aac84e0df4da2b147fca70cf8255,1,16.89,180.00,196.89,1.0,2017-11-14 19:45:42,2017-11-14 19:45:42
3,00053a61a98854899e70ed204dd4bafe,1,37.18,382.00,419.18,2.0,2018-02-28 11:15:41,2018-02-28 11:15:41
4,0005e1862207bf6ccc02e4228effd9a0,1,15.12,135.00,150.12,1.0,2017-03-04 23:32:12,2017-03-04 23:32:12
...,...,...,...,...,...,...,...,...
58240,fffbf87b7a1a6fa8b03f081c5f51a201,1,18.32,149.00,167.32,1.0,2017-12-27 22:36:41,2017-12-27 22:36:41
58241,fffcf5a5ff07b0908bd4e2dbc735a684,1,497.42,1570.00,2067.42,2.0,2017-06-08 21:00:36,2017-06-08 21:00:36
58242,fffea47cd6d3cc0a88bd621562a9d061,1,19.69,64.89,84.58,1.0,2017-12-10 20:07:56,2017-12-10 20:07:56
58243,ffff371b4d645b6ecea244b27531430a,1,22.56,89.90,112.46,1.0,2017-02-07 15:49:16,2017-02-07 15:49:16


In [20]:
data_ord_cus_1['recent']=(date_12-data_ord_cus_1['last']).dt.days

data_ord_cus_1

,customer_unique_id,n_achats,fright_value,price,total_price,n_article,first,last,recent
0,0000f46a3911fa3c0805444483337064,1,17.22,69.00,86.22,1.0,2017-03-10 21:05:03,2017-03-10 21:05:03,358
1,0000f6ccb0745a6a4b88665a16c9f078,1,17.63,25.99,43.62,1.0,2017-10-12 20:29:41,2017-10-12 20:29:41,142
2,0004aac84e0df4da2b147fca70cf8255,1,16.89,180.00,196.89,1.0,2017-11-14 19:45:42,2017-11-14 19:45:42,109
3,00053a61a98854899e70ed204dd4bafe,1,37.18,382.00,419.18,2.0,2018-02-28 11:15:41,2018-02-28 11:15:41,3
4,0005e1862207bf6ccc02e4228effd9a0,1,15.12,135.00,150.12,1.0,2017-03-04 23:32:12,2017-03-04 23:32:12,364
...,...,...,...,...,...,...,...,...,...
58240,fffbf87b7a1a6fa8b03f081c5f51a201,1,18.32,149.00,167.32,1.0,2017-12-27 22:36:41,2017-12-27 22:36:41,66
58241,fffcf5a5ff07b0908bd4e2dbc735a684,1,497.42,1570.00,2067.42,2.0,2017-06-08 21:00:36,2017-06-08 21:00:36,268
58242,fffea47cd6d3cc0a88bd621562a9d061,1,19.69,64.89,84.58,1.0,2017-12-10 20:07:56,2017-12-10 20:07:56,83
58243,ffff371b4d645b6ecea244b27531430a,1,22.56,89.90,112.46,1.0,2017-02-07 15:49:16,2017-02-07 15:49:16,389


In [21]:

data_ord_cus_1['frequence']=((date_12-data_ord_cus_1['first']).dt.days)/(data_ord_cus_1['n_achats'])

In [22]:
data_ord_cus_1[data_ord_cus_1['n_achats']>1]

,customer_unique_id,n_achats,fright_value,price,total_price,n_article,first,last,recent,frequence
61,004288347e5e88a27ded2bb23747066c,2,18.240000,158.945000,177.185000,1.0,2017-07-27 14:13:03,2018-01-14 07:36:54,48,109.500000
184,00cc12a6d8b578b8ebd21ea4e2ae8b27,2,13.200000,49.900000,63.100000,1.0,2017-03-21 19:25:22,2017-03-21 19:25:23,347,173.500000
254,011b4adcd54683b480c4d841250a987f,2,26.435000,91.715000,118.150000,1.5,2017-08-22 12:51:29,2018-02-15 11:40:57,16,96.500000
286,013f4353d26bb05dc6652f1269458d8d,2,39.745000,138.450000,178.195000,1.5,2017-11-24 13:33:20,2017-11-28 13:30:58,95,49.500000
305,015557c9912277312b9073947804a7ba,2,12.610000,144.950000,157.560000,1.0,2017-03-23 22:45:46,2017-05-01 14:48:33,306,172.500000
...,...,...,...,...,...,...,...,...,...,...
57956,fed2005ccab4fcf1a40ebdaff032a148,2,20.240000,30.310000,50.550000,1.5,2017-06-17 17:04:27,2017-06-17 17:04:31,259,129.500000
58068,ff44401d0d8f5b9c54a47374eb48c1b8,2,14.100000,19.900000,34.000000,1.0,2017-05-19 21:20:54,2017-05-19 21:20:54,288,144.000000
58135,ff8892f7c26aa0446da53d01b18df463,2,26.095000,138.975000,165.070000,1.0,2017-05-24 16:09:14,2017-11-26 23:25:43,97,141.500000
58150,ff922bdd6bafcdf99cb90d7f39cea5b3,3,14.936667,31.596667,46.533333,1.0,2017-02-22 12:26:42,2017-09-14 14:24:04,170,124.666667


In [23]:
data_ord_cus_1.drop(columns=['first','last'],inplace=True)
data_ord_cus_1

,customer_unique_id,n_achats,fright_value,price,total_price,n_article,recent,frequence
0,0000f46a3911fa3c0805444483337064,1,17.22,69.00,86.22,1.0,358,358.0
1,0000f6ccb0745a6a4b88665a16c9f078,1,17.63,25.99,43.62,1.0,142,142.0
2,0004aac84e0df4da2b147fca70cf8255,1,16.89,180.00,196.89,1.0,109,109.0
3,00053a61a98854899e70ed204dd4bafe,1,37.18,382.00,419.18,2.0,3,3.0
4,0005e1862207bf6ccc02e4228effd9a0,1,15.12,135.00,150.12,1.0,364,364.0
...,...,...,...,...,...,...,...,...
58240,fffbf87b7a1a6fa8b03f081c5f51a201,1,18.32,149.00,167.32,1.0,66,66.0
58241,fffcf5a5ff07b0908bd4e2dbc735a684,1,497.42,1570.00,2067.42,2.0,268,268.0
58242,fffea47cd6d3cc0a88bd621562a9d061,1,19.69,64.89,84.58,1.0,83,83.0
58243,ffff371b4d645b6ecea244b27531430a,1,22.56,89.90,112.46,1.0,389,389.0


In [24]:
data_ord_cus_1.to_csv("data_ord_1.csv",index=False)

In [25]:
def data_enr(data,date_f,nom_fichier):
    """
    cette fonction prépare la base de données en calculant la moyenne des dépenses, et les fréquences d'achat
    
    
    :param data: dataframe des données brut suivant les plages sélectionnées
    :param date_f:date de fin 
    :param nom_fichier: le nom du fichier d'enregistrement
    
    """
    data_ord_cus=data.groupby('customer_unique_id', as_index=False).agg(n_achats=('order_id','count'),
                                                                        fright_value =('fright_value','mean'),
                                                                        price = ('price','mean'),
                                                                        total_price = ('total_price','mean'),
                                                                        n_article =('n_article','mean'),
                                                                        first=('order_purchase_timestamp','min'),
                                                                        last=('order_purchase_timestamp','max')
                                                                        )
    data_ord_cus['recent']=(date_f-data_ord_cus['last']).dt.days
    data_ord_cus['frequence']=((date_f-data_ord_cus['first']).dt.days)/(data_ord_cus['n_achats'])
    data_ord_cus.drop(columns=['first','last'],inplace=True)
    data_ord_cus.to_csv(nom_fichier,index=False)
    

In [26]:
data_enr(data=data_2,date_f=date_22,nom_fichier = "data_ord_2.csv")
data_enr(data=data_3,date_f=date_32,nom_fichier = "data_ord_3.csv")
data_enr(data=data_4,date_f=date_42,nom_fichier = "data_ord_4.csv")
data_enr(data=data_5,date_f=date_52,nom_fichier = "data_ord_5.csv")

In [27]:
data_test=pd.read_csv("data_ord_5.csv")
data_test

,customer_unique_id,n_achats,fright_value,price,total_price,n_article,recent,frequence
0,0000f46a3911fa3c0805444483337064,1,17.22,69.00,86.22,1.0,419,419.0
1,0000f6ccb0745a6a4b88665a16c9f078,1,17.63,25.99,43.62,1.0,203,203.0
2,0004aac84e0df4da2b147fca70cf8255,1,16.89,180.00,196.89,1.0,170,170.0
3,0004bd2a26a76fe21f786e4fbd80607f,1,12.98,154.00,166.98,1.0,28,28.0
4,00050ab1314c0e55a6ca13cf7181fecf,1,7.39,27.99,35.38,1.0,13,13.0
...,...,...,...,...,...,...,...,...
71702,fffcf5a5ff07b0908bd4e2dbc735a684,1,497.42,1570.00,2067.42,2.0,329,329.0
71703,fffea47cd6d3cc0a88bd621562a9d061,1,19.69,64.89,84.58,1.0,144,144.0
71704,ffff371b4d645b6ecea244b27531430a,1,22.56,89.90,112.46,1.0,450,450.0
71705,ffff5962728ec6157033ef9805bacc48,1,18.69,115.00,133.69,1.0,1,1.0
